<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Hypothesis-Testing-with-A/B-Testing" data-toc-modified-id="Hypothesis-Testing-with-A/B-Testing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Hypothesis Testing with A/B Testing</a></span></li><li><span><a href="#The-data" data-toc-modified-id="The-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>The data</a></span><ul class="toc-item"><li><span><a href="#Ensuring-all-variants-of-the-test-have-the-same-number-of-entries" data-toc-modified-id="Ensuring-all-variants-of-the-test-have-the-same-number-of-entries-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Ensuring all variants of the test have the same number of entries</a></span></li></ul></li><li><span><a href="#Test-1---Chi-Square" data-toc-modified-id="Test-1---Chi-Square-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Test 1 - Chi-Square</a></span><ul class="toc-item"><li><span><a href="#Result---And-an-error" data-toc-modified-id="Result---And-an-error-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Result - And an error</a></span></li><li><span><a href="#A-function-to-return-the-significance-level" data-toc-modified-id="A-function-to-return-the-significance-level-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>A function to return the significance level</a></span></li></ul></li><li><span><a href="#Test-2---Binomial-Test" data-toc-modified-id="Test-2---Binomial-Test-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Test 2 - Binomial Test</a></span><ul class="toc-item"><li><span><a href="#Calculating-required-success-rates-(probabilities)" data-toc-modified-id="Calculating-required-success-rates-(probabilities)-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Calculating required success rates (probabilities)</a></span></li></ul></li><li><span><a href="#Calculating-the-p-values" data-toc-modified-id="Calculating-the-p-values-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Calculating the p-values</a></span><ul class="toc-item"><li><span><a href="#Significance-for-group-A" data-toc-modified-id="Significance-for-group-A-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Significance for group A</a></span></li><li><span><a href="#Significance-for-group-B" data-toc-modified-id="Significance-for-group-B-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Significance for group B</a></span></li><li><span><a href="#Significance-for-group-C" data-toc-modified-id="Significance-for-group-C-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Significance for group C</a></span></li></ul></li><li><span><a href="#Result" data-toc-modified-id="Result-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Result</a></span></li></ul></div>

In [31]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from scipy.stats import binom_test

# Hypothesis Testing with A/B Testing

This project focuses on an A/B test for a gaming company. The company is looking to get users to buy an upgrade package but do not know how much to charge for it. As such, they have tested it using three different price points:

* Group A - upgrade for **£0.99**
* Group B - upgrade for **£1.99**
* Group C - upgrade for **£4.99**

The aim is to generate at least £1,000 per week from people choosing to upgrade.

Hypothesis testing will be used to determine which price point is the best to achieve the goal.

# The data

In [32]:
# Read in the clicks.csv file
abdata = pd.read_csv('clicks.csv')
print(abdata.head())

    user_id group is_purchase
0  8e27bf9a     A          No
1  eb89e6f0     A          No
2  7119106a     A          No
3  e53781ff     A          No
4  02d48cf1     A         Yes


In [33]:
abdata.shape

(4998, 3)

## Ensuring all variants of the test have the same number of entries

In [34]:
group_a = abdata[abdata['group'] == 'A']
group_b = abdata[abdata['group'] == 'B']
group_c = abdata[abdata['group'] == 'C']

In [35]:
group_a.shape, group_b.shape, group_c.shape

((1666, 3), (1666, 3), (1666, 3))

# Test 1 - Chi-Square

In [36]:
# Creating a contingency table with pd.crosstab
Xtab = pd.crosstab(abdata.group, abdata.is_purchase)
Xtab

is_purchase,No,Yes
group,,
A,1350,316
B,1483,183
C,1583,83


In [37]:
# Calculate the p-value using a Chi-Square test
chi2, pval, dof, expected = chi2_contingency(Xtab)
pval

2.4126213546684264e-35

## Result - And an error

The Chi-Square test is inappropriate to use in this scenario because it is testing the distributions of Yes/No outcomes for each group, i.e. it does not account for the probabilities of each outcome for each group.

It is extremely probable that many more users will buy the upgrade when it is priced at £0.99 vs £4.99.

## A function to return the significance level

In [38]:
def is_significant(pvalue):
    if pvalue < 0.05 and pvalue > 0:
        return True
    elif pvalue >= 0.05 and pvalue < 1:
        return False
    else:
        print('There is an error with the pvalue; please make sure it is numeric, and between zero and 1')
    return None

In [39]:
print(is_significant(pval))

True


# Test 2 - Binomial Test

Whether a user chooses to buy the upgrade or not is binary, and so a binomial test is suitable.

However, this tests requires the probability of "success" vs "failure" is known.

These probabilities can be calculated for each of the test variants using the required success rate to satisfy the £1000/week requirement at each price point.

## Calculating required success rates (probabilities)

We have 1666 customers in each group, and we know the amount of money must be made per week at a minimum to be considered successful i.e. £1000.

In [40]:
# finding the number of visits
num_visits = len(abdata)
num_visits

4998

In [62]:
# Calculating the purchase rate needed at 0.99 to satisfy the £1000/week minimum target
num_sales_needed_099 = 1000/0.99
p_sales_needed_099 = num_sales_needed_099/num_visits
p_sales_needed_099 # this is the required probability of success for this variant

0.20210104243717691

In [42]:
# Calculating the purchase rate needed at £1.99
num_sales_needed_199 = 1000/1.99
p_sales_needed_199 = num_sales_needed_199/num_visits
p_sales_needed_199

0.10054272965467594

In [43]:
# Calculating the purchase rate needed at £4.99
num_sales_needed_499 = 1000/4.99
p_sales_needed_499 = num_sales_needed_499/num_visits
p_sales_needed_499

0.040096198800161346

In [44]:
# Calculating sample size & sales for £0.99 price point
samp_size_099 = np.sum(abdata.group == 'A')
sales_099 = np.sum((abdata.group == 'A') & (abdata.is_purchase == 'Yes'))

In [45]:
print(samp_size_099)
print(sales_099)

1666
316


In [46]:
# Calculating sample size & sales for £1.99 price point
samp_size_199 = np.sum(abdata.group == 'B')
sales_199 = np.sum((abdata.group == 'B') & (abdata.is_purchase == 'Yes'))

In [47]:
print(samp_size_199)
print(sales_199)

1666
183


In [48]:
# Calculating sample size & sales for £4.99 price point
samp_size_499 = np.sum(abdata.group == 'C')
sales_499 = np.sum((abdata.group == 'C') & (abdata.is_purchase == 'Yes'))

In [49]:
print(samp_size_499)
print(sales_499)

1666
83


# Calculating the p-values

Now that we have each of the required success rates, we can put them into the binomial test function and calculate the p-values for each variant of the test given the actual sales made in each group.

**The null hypothesis**:

The results seen are the maximum conversions that will occur for each price point. 

**The alternative hypothesis**: 

In reality, a greater conversion rate will be seen for each price point.

## Significance for group A

In [50]:
pvalueA = binom_test(sales_099, n=samp_size_099, p=p_sales_needed_099, alternative='greater')
pvalueA

0.9028081076188985

## Significance for group B

In [51]:
pvalueB = binom_test(sales_199, n=samp_size_199, p=p_sales_needed_199, alternative='greater')
pvalueB

0.11184562623739903

## Significance for group C

In [52]:
pvalueC = binom_test(sales_499, n=samp_size_499, p=p_sales_needed_499, alternative='greater')
pvalueC

0.027944826659907135

# Result

In [53]:
print(is_significant(pvalueA))
print(is_significant(pvalueB))
print(is_significant(pvalueC))

False
False
True


In [54]:
Xtab

is_purchase,No,Yes
group,,
A,1350,316
B,1483,183
C,1583,83


In [61]:
revenue_a = Xtab['Yes'][0] * 0.99
revenue_b = Xtab['Yes'][0] * 1.99
revenue_c = Xtab['Yes'][0] * 4.99
print("money generated from variant A: £{}".format(revenue_a))
print("money generated from variant B: £{}" .format(revenue_b))
print("money generated from variant C: £{}" .format(revenue_c))

money generated from variant A: £312.84
money generated from variant B: £628.84
money generated from variant C: £1576.8400000000001


So, group C has the best price point to maximise profits for the company. It is the only one for which we can reject the null hypothesis and accept the alternative hypothesis - that in reality, we can expect more customers to buy the upgrade than did in the test.

Group C already made the most money by more than a factor of 2, so when this is combined with the result of the hypothesis test it is the clear winner.